In [1]:
import pandas as pd

data = pd.read_csv("../data/AAPL_backtest.csv", index_col=0, parse_dates=True)
data.tail(3)


,Open,High,Low,Close,Volume,SMA_10,SMA_50,SMA_200,Volatility_30,RSI_14,BB_MID,BB_UPPER,BB_LOWER,ATR_14,Signal,Position,Return,Strategy_Return
Price,,,,,,,,,,,,,,,,,,
2024-12-26,257.276679,259.179926,256.718662,258.103729,27237100.0,251.649666,235.495564,211.097456,0.008575,76.452850,246.616033,259.723060,233.509006,4.107561,1,1.0,0.003176,0.003176
2024-12-27,256.917949,257.784897,252.164833,254.685883,42355300.0,252.409964,235.975156,211.521336,0.009190,67.626275,247.645379,259.813298,235.477460,4.350269,1,1.0,-0.013242,-0.013242
2024-12-30,251.337769,252.603281,249.863009,251.307877,35557500.0,252.815524,236.379822,211.919042,0.009546,60.225586,248.386249,259.448570,237.323927,4.304004,1,1.0,-0.013263,-0.013263


In [2]:
#This checks metrics and gives feedback.

import numpy as np

def compute_metrics(df):
    strat_curve = (1 + df["Strategy_Return"]).cumprod()
    years = (df.index[-1] - df.index[0]).days / 365.25
    cagr = strat_curve.iloc[-1]**(1/years) - 1
    vol = df["Strategy_Return"].std() * np.sqrt(252)
    sharpe = (df["Strategy_Return"].mean() * 252) / (df["Strategy_Return"].std() * np.sqrt(252))
    roll_max = strat_curve.cummax()
    dd = (strat_curve / roll_max - 1).min()
    return {"CAGR": cagr, "Volatility": vol, "Sharpe": sharpe, "MaxDD": dd}

metrics = compute_metrics(data)
metrics



{'CAGR': np.float64(0.04537456477983004),
 'Volatility': np.float64(0.16433131022232675),
 'Sharpe': np.float64(0.3546885590564063),
 'MaxDD': np.float64(-0.08671305577045663)}

In [3]:
def reflect_and_replan(metrics, sharpe_min=1.0, max_dd=-0.25):
    decisions = []
    if metrics["Sharpe"] < sharpe_min:
        decisions.append("Sharpe too low — consider reducing risk exposure or adding filters.")
    if metrics["MaxDD"] < max_dd:
        decisions.append("Drawdown exceeded limit — strategy too aggressive.")
    if not decisions:
        decisions.append("Strategy acceptable. No changes needed.")
    return decisions

reflection_notes = reflect_and_replan(metrics)
reflection_notes


['Sharpe too low — consider reducing risk exposure or adding filters.']

In [4]:
import os
from datetime import datetime

os.makedirs("../logs", exist_ok=True)
with open("../logs/reflection_day5.txt", "w") as f:
    f.write(f"Reflection run {datetime.now()}\n")
    f.write(f"Metrics: {metrics}\n")
    for note in reflection_notes:
        f.write(f"- {note}\n")

print("Reflection notes saved to /logs/reflection_day5.txt")


Reflection notes saved to /logs/reflection_day5.txt


In [5]:
summary = f"""
Reflection Report ({datetime.today().date()}):
- CAGR: {metrics['CAGR']:.2%}
- Volatility: {metrics['Volatility']:.2%}
- Sharpe: {metrics['Sharpe']:.2f}
- Max Drawdown: {metrics['MaxDD']:.2%}

Notes:
{'; '.join(reflection_notes)}
"""

print(summary)



Reflection Report (2025-09-16):
- CAGR: 4.54%
- Volatility: 16.43%
- Sharpe: 0.35
- Max Drawdown: -8.67%

Notes:
Sharpe too low — consider reducing risk exposure or adding filters.

